In [ ]:
import psycopg2
import numpy as np
import pandas as pd
from sklearn.preprocessing import MaxAbsScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv1D, SpatialDropout1D, LSTM, Dense, Dropout, LayerNormalization, Input
from tensorflow.keras.layers import MultiHeadAttention
from tensorflow.keras.regularizers import l2, l1_l2,l1
import keras_tuner as kt
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import Callback

#Connects to the PostgreSQL database and retrieves the historical price data of a specific cryptocurrency.
#Returns a DataFrame with the data sorted by date.
def fetch_data(ticker, conn_params):
    conn = psycopg2.connect(**conn_params)
    query = f"""
    SELECT * FROM crypto_prices
    WHERE ticker = '{ticker}'
    ORDER BY date;
    """
    df = pd.read_sql_query(query, conn)
    conn.close()
    return df

#Converts the date column into a time index, selects relevant features,
#and scales them using MaxAbsScaler to normalize the values.
def preprocess_data(df):
    df['date'] = pd.to_datetime(df['date'])
    df.set_index('date', inplace=True)
    features = df[['features']] 
    
    scaler = MaxAbsScaler()
    scaled_features = scaler.fit_transform(features)
    return scaled_features
    
#Generates training sets using a defined time window, time_step, to predict
#the closing price (column 3) based on previous data.
def create_dataset(data, time_step=12):
    X, y = [], []
    for i in range(len(data) - time_step - 1):
        X.append(data[i:(i + time_step), :])
        y.append(data[i + time_step, 3]) 
    return np.array(X), np.array(y)

#Calculates the sMAPE (Symmetric Mean Absolute Percentage Error) metric
#to evaluate the mean percentage error between predicted and actual values.
def smape(y_true, y_pred):
    numerator = tf.abs(y_true - y_pred)
    denominator = (tf.abs(y_true) + tf.abs(y_pred)) / 2
    smape_value = tf.where(tf.equal(denominator, 0), tf.zeros_like(numerator), numerator / denominator)
    return tf.reduce_mean(smape_value) * 100  # Convert to percentage

#Defines the MAE (Mean Absolute Error) loss metric.
def mean_absolute_error_tf(y_true, y_pred):
    return tf.reduce_mean(tf.abs(y_true - y_pred))


#Custom callback that reduces the L2 regularization value in specific layers at the end of each epoch,
#adjusting the L2 penalty to avoid overfitting
class L2DecayCallback(Callback):
    def __init__(self, initial_l2, decay_rate, target_layers):
        super().__init__()
        self.initial_l2 = initial_l2
        self.decay_rate = decay_rate
        self.target_layers = target_layers

    def on_epoch_end(self, epoch, logs=None):
        current_l2 = self.initial_l2 * (self.decay_rate ** epoch)
        for layer in self.target_layers:
            if hasattr(layer, 'kernel_regularizer'):
                layer.kernel_regularizer.l2 = current_l2

#Builds a sequential model with Conv1D, LSTM layers, and a multi-head attention layer,
#with L2 regularization that decays during training.
#Uses Spatial Dropout to reduce overfitting and L1 regularization in the multi-head attention layer
#to improve feature selection.
#Utilizes Keras Tuner to optimize hyperparameters like the number of filters and learning rate.
def build_model_with_decay(hp, X_train, initial_l2, decay_rate):
    # Definición de la entrada
    inputs = Input(shape=(X_train.shape[1], X_train.shape[2]))
    
    # Conv1D layer with Spatial Dropout
    conv_layer = Conv1D(
        filters=hp.Int('filters', min_value=32, max_value=128, step=32),
        kernel_size=3,
        activation='relu',
        kernel_regularizer=l2(initial_l2)
    )(inputs)
    conv_layer = SpatialDropout1D(rate=0.2)(conv_layer)  # Add Spatial Dropout

    # LSTM layer with Spatial Dropout
    lstm_layer = LSTM(
        units=hp.Int('units_layer_1', min_value=50, max_value=500, step=50),
        return_sequences=True,
        kernel_regularizer=l2(initial_l2)
    )(conv_layer)
    lstm_layer = SpatialDropout1D(rate=0.2)(lstm_layer)  # Add Spatial Dropout

    # multi-head attention layer
    attention_layer = MultiHeadAttention(
        num_heads=hp.Int('num_heads', min_value=2, max_value=8, step=1),
        key_dim=hp.Int('key_dim', min_value=8, max_value=64, step=8),
        kernel_regularizer=l1(0.01)
    )(lstm_layer, lstm_layer)  # Applies attention to the output of the LSTM layer.

    # Normalization after the attention layer.
    attention_output = LayerNormalization()(attention_layer)

    # Output layer
    output = Dense(1, kernel_regularizer=l2(initial_l2))(attention_output)

    # Build model
    model = Model(inputs=inputs, outputs=output)

    # Compile model
    model.compile(optimizer=tf.keras.optimizers.SGD(
        learning_rate=hp.Float('learning_rate', min_value=1e-5, max_value=1e-2, sampling='LOG')
    ), loss='mean_squared_error')

    return model 


#Stores error metrics in a database table, allowing for the analysis and 
# comparison of the performance of different models.
def store_errors(ticker, mse, mae, r2, mape, smape_val, conn_params):
    mse = float(mse)
    mae = float(mae)
    r2 = float(r2)
    mape = float(mape)
    smape_val = float(smape_val)
    
    conn = psycopg2.connect(**conn_params)
    cursor = conn.cursor()
    
    query = """
    INSERT INTO error (ticker, mse, mae, r2, mape, smape, coment)
    VALUES (%s, %s, %s, %s, %s, %s, ' ');
    """
    cursor.execute(query, (ticker, mse, mae, r2, mape, smape_val))
    
    conn.commit()
    cursor.close()
    conn.close()

#Performs the full cycle of training and evaluation for each cryptocurrency in the list of tickers.
#Splits the data into training and validation sets, uses Keras Tuner for hyperparameter optimization via Bayesian search,
#evaluates the model with performance metrics, and saves the best model to a file.
def train_and_evaluate(tickers, conn_params, time_step=48):
    for ticker in tickers:
        print(f"Processing ticker: {ticker}")

        df = fetch_data(ticker, conn_params)
        scaled_features = preprocess_data(df)
        X, y = create_dataset(scaled_features, time_step)
        
        split_idx = int(len(X) * 0.8)
        X_train, X_val = X[:split_idx], X[split_idx:]
        y_train, y_val = y[:split_idx], y[split_idx:]

        #Defines the tuner using Bayesian Optimization.
        tuner = kt.BayesianOptimization(
            lambda hp: build_model_with_decay(hp,
            X_train,
            0.01,
            decay_rate=0.9 
            ),
            objective='val_loss',
            max_trials=50,  
            directory='my_dir',
            project_name=f'crypto_lstm_{ticker}'
        )

        # Performs hyperparameter search.
        fixed_batch_size = 128
        tuner.search(X_train, y_train, epochs=150, validation_split=0.2, 
                     batch_size=fixed_batch_size,
                     callbacks=[EarlyStopping(monitor='val_loss', patience=3)])

        # Evaluate the best model
        best_model = tuner.get_best_models(num_models=1)[0]
        y_pred = best_model.predict(X_val)
        
        mse = mean_squared_error(y_val, y_pred)
        mae = mean_absolute_error(y_val, y_pred)
        r2 = r2_score(y_val, y_pred)
        mape = np.mean(np.abs((y_val - y_pred) / y_val)) * 100

        smape_val = smape(tf.convert_to_tensor(y_val, dtype=tf.float32), 
                          tf.convert_to_tensor(y_pred, dtype=tf.float32)).numpy()
        
        print(f"Ticker: {ticker}, MSE: {mse}, MAE: {mae}, R²: {r2}, MAPE: {mape}, sMAPE: {smape_val}")

        #Save the best model
        model_path = f"model_{ticker}.keras"
        best_model.save(model_path)
        print(f"Model saved to {model_path}")
        
        store_errors(ticker, mse, mae, r2, mape, smape_val, conn_params)

# DB Params
conn_params = {
    'dbname': 'dbname',
    'user': 'user',
    'password': 'password',
    'host': 'localhost',
    'port': 'port'
}
tickers = [
"BTCUSDT" #Bitcoin
]
train_and_evaluate(tickers, conn_params)


In [ ]:
import psycopg2
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MaxAbsScaler

#Connects to the PostgreSQL database and retrieves historical 
# price data of a specific cryptocurrency, ordered by date.
def fetch_data(ticker, conn_params):
    conn = psycopg2.connect(**conn_params)
    query = f"""
    SELECT * FROM crypto_prices
    WHERE ticker = '{ticker}'
    ORDER BY date;
    """
    df = pd.read_sql_query(query, conn)
    conn.close()
    return df

#Retrieves the timestep value from the "time1" table in the database for a given ticker.
def fetch_timestep(ticker, conn_params):
    conn = psycopg2.connect(**conn_params)
    query = f"""
    SELECT timestep FROM timestep
    WHERE ticker = '{ticker}';
    """
    cursor = conn.cursor()
    cursor.execute(query)
    timestep = cursor.fetchone()[0]  
    conn.close()
    return int(timestep)  # Convert timestep to int


#Preprocesses the data by converting dates, scaling features, and
#returning the last actual price along with the scaler used.
def preprocess_data(df):
    df['date'] = pd.to_datetime(df['date'])
    df.set_index('date', inplace=True)
    features = df[['features']]
    
    # Apply scaling.
    scaler = MaxAbsScaler()
    scaled_features = scaler.fit_transform(features)

    return scaled_features, features['close'].values[-1], scaler  

#Loads a previously saved model in Keras format, customized with the sMAPE and MAE metrics.
def load_model(ticker):
    model_path = f"modelo_{ticker}.keras"
    return tf.keras.models.load_model(model_path, custom_objects={"smape": smape, "mean_absolute_error_tf": mean_absolute_error_tf})

# Defines the method to correctly load a saved model with custom metrics.
def smape(y_true, y_pred):
    numerator = tf.abs(y_true - y_pred)
    denominator = (tf.abs(y_true) + tf.abs(y_pred)) / 2
    return tf.reduce_mean(numerator / denominator) * 100  

# Defines the method to correctly load a saved model with custom metrics.
def mean_absolute_error_tf(y_true, y_pred):
    return tf.reduce_mean(tf.abs(y_true - y_pred))

#Calculates the percentage of profit or loss based on predicted and actual prices, ensuring no division by zero occurs.
def calculate_profit_percentage(predicted_price, actual_price):
    if actual_price != 0:
        return ((predicted_price - actual_price) / actual_price) * 100
    return 0  # Prevents division by zero.

# Inserts the profit or loss percentage into the "profit" table of the database.
def store_profit(ticker, profit, conn_params):
    conn = psycopg2.connect(**conn_params)
    cursor = conn.cursor()
    
    query = """
    INSERT INTO profit (ticker, profit)
    VALUES (%s, %s);
    """
    cursor.execute(query, (ticker, profit))
    
    conn.commit()
    cursor.close()
    conn.close()


#Performs the prediction process for a list of tickers, preprocesses data, loads the model, predicts the next price,
#calculates the profit or loss, and stores the results in the database.
def prediction_module(tickers, conn_params):
    for ticker in tickers:
        print(f"Processing ticker: {ticker}")

    
        df = fetch_data(ticker, conn_params)
        scaled_features, actual_price, scaler = preprocess_data(df)

        time_step = fetch_timestep(ticker, conn_params)

        
        #Makes predictions for the next interval using the timestep obtained.
        if len(scaled_features) >= time_step:  
            last_data = scaled_features[-time_step:].reshape(1, time_step, scaled_features.shape[1])  
        else:
            print(f"Not enough data to make a prediction for ticker: {ticker}")
            continue

        # Load saved model
        model = load_model(ticker)

        # Make prediction
        predicted_price_scaled = model.predict(last_data)[0, 0]
        predicted_price = scaler.inverse_transform(np.array([[predicted_price_scaled] + [0] * (scaled_features.shape[1] - 1)])[0].reshape(1, -1))[0][0]

        # Calculates the profit or loss based on the predicted price and the actual price.
        profit_percentage = calculate_profit_percentage(predicted_price, actual_price)

        # Print results
        print(f"Ticker: {ticker}, Predicted Price: {predicted_price}, Actual Price: {actual_price}, Profit/Loss: {profit_percentage}%")

        # Store profit/loss on the DB
        store_profit(ticker, profit_percentage, conn_params)

# DB Params
conn_params = {
    'dbname': 'crypto8',
    'user': 'postgres',
    'password': 'loquesea',
    'host': 'localhost',
    'port': '5432'
}
tickers = [
"BTCSDT" #Bitcoin
]  # Tickers list

prediction_module(tickers, conn_params) #Run